# 如何从工具返回工件

```{=mdx}
:::info 前提条件
本指南假定您已熟悉以下概念：

- [ToolMessage](/docs/concepts/messages/#toolmessage)
- [Tools](/docs/concepts/tools)
- [工具调用](/docs/concepts/tool_calling)

:::
```

工具是可以由模型调用的实用程序，其输出设计用于反馈给模型。然而，有时工具执行过程中会产生一些工件，我们希望这些工件可以被链或代理中的下游组件访问，但不希望将其暴露给模型本身。

例如，如果一个工具返回类似自定义对象或图像的内容，我们可能希望向模型传递有关该输出的一些元数据，而不是将实际输出传递给模型。同时，我们可能希望在其他地方（例如下游工具中）能够访问到这个完整输出。

Tool 和 [ToolMessage](https://api.js.langchain.com/classes/langchain_core.messages_tool.ToolMessage.html) 接口使我们能够区分工具输出中面向模型的部分（即 `ToolMessage.content`）和面向模型外部使用的部分（即 `ToolMessage.artifact`）。

```{=mdx}
:::caution 兼容性

此功能需要 `@langchain/core>=0.2.16`。请参阅[安装指南](/docs/how_to/installation/#installing-integration-packages)了解如何升级。

:::
```

## 定义工具

如果我们希望工具能够区分消息内容和其他工件，则在定义工具时需要指定 `response_format: "content_and_artifact"`，并确保返回的是 [`content`, `artifact`] 形式的元组：

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";

const randomIntToolSchema = z.object({
  min: z.number(),
  max: z.number(),
  size: z.number(),
});

const generateRandomInts = tool(async ({ min, max, size }) => {
  const array: number[] = [];
  for (let i = 0; i < size; i++) {
    array.push(Math.floor(Math.random() * (max - min + 1)) + min);
  }
  return [
    `Successfully generated array of ${size} random ints in [${min}, ${max}].`,
    array,
  ];
}, {
  name: "generateRandomInts",
  description: "Generate size random ints in the range [min, max].",
  schema: randomIntToolSchema,
  responseFormat: "content_and_artifact",
});

## 使用 ToolCall 调用工具

如果我们仅通过工具参数直接调用工具，你会发现我们只能获得 `Tool` 输出中的内容部分：

In [3]:
await generateRandomInts.invoke({min: 0, max: 9, size: 10});

Successfully generated array of 10 random ints in [0, 9].


为了同时获取内容和工件，我们需要使用一个`ToolCall`（它只是一个包含`"name"`、`"args"`、`"id"`和`"type"`键的字典）来调用模型，该字典包含生成像工具调用ID这样的ToolMessage所需的附加信息：

In [4]:
await generateRandomInts.invoke(
  {
    name: "generate_random_ints",
    args: {min: 0, max: 9, size: 10},
    id: "123", // Required
    type: "tool_call", // Required
  }
);

ToolMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'Successfully generated array of 10 random ints in [0, 9].',
    artifact: [
      0, 6, 5, 5, 7,
      0, 6, 3, 7, 5
    ],
    tool_call_id: '123',
    name: 'generateRandomInts',
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ 'langchain_core', 'messages' ],
  content: 'Successfully generated array of 10 random ints in [0, 9].',
  name: 'generateRandomInts',
  additional_kwargs: {},
  response_metadata: {},
  id: undefined,
  tool_call_id: '123',
  artifact: [
    0, 6, 5, 5, 7,
    0, 6, 3, 7, 5
  ]
}


## 与模型一起使用

使用一个[工具调用模型](/docs/how_to/tool_calling/)，我们可以轻松地使用模型调用我们的工具并生成工具消息：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>
```

In [7]:
const llmWithTools = llm.bindTools([generateRandomInts])

const aiMessage = await llmWithTools.invoke("generate 6 positive ints less than 25")
aiMessage.tool_calls

[
  {
    name: 'generateRandomInts',
    args: { min: 1, max: 24, size: 6 },
    id: 'toolu_019ygj3YuoU6qFzR66juXALp',
    type: 'tool_call'
  }
]


In [8]:
await generateRandomInts.invoke(aiMessage.tool_calls[0])

ToolMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'Successfully generated array of 6 random ints in [1, 24].',
    artifact: [ 18, 20, 16, 15, 17, 19 ],
    tool_call_id: 'toolu_019ygj3YuoU6qFzR66juXALp',
    name: 'generateRandomInts',
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ 'langchain_core', 'messages' ],
  content: 'Successfully generated array of 6 random ints in [1, 24].',
  name: 'generateRandomInts',
  additional_kwargs: {},
  response_metadata: {},
  id: undefined,
  tool_call_id: 'toolu_019ygj3YuoU6qFzR66juXALp',
  artifact: [ 18, 20, 16, 15, 17, 19 ]
}


如果我们只传入工具调用参数，我们将只能返回内容：

In [9]:
await generateRandomInts.invoke(aiMessage.tool_calls[0]["args"])

Successfully generated array of 6 random ints in [1, 24].


如果我们想要声明式地创建一个链，我们可以这样做：

In [10]:
const extractToolCalls = (aiMessage) => aiMessage.tool_calls;

const chain = llmWithTools.pipe(extractToolCalls).pipe(generateRandomInts.map());

await chain.invoke("give me a random number between 1 and 5");

[
  ToolMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: 'Successfully generated array of 1 random ints in [1, 5].',
      artifact: [Array],
      tool_call_id: 'toolu_01CskofJCQW8chkUzmVR1APU',
      name: 'generateRandomInts',
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ 'langchain_core', 'messages' ],
    content: 'Successfully generated array of 1 random ints in [1, 5].',
    name: 'generateRandomInts',
    additional_kwargs: {},
    response_metadata: {},
    id: undefined,
    tool_call_id: 'toolu_01CskofJCQW8chkUzmVR1APU',
    artifact: [ 1 ]
  }
]


## 相关内容

你现在已经了解了如何从工具调用中返回额外的产物。

以下指南可能也会引起你的兴趣：

- [创建自定义工具](/docs/how_to/custom_tools)
- [使用 LangGraph 构建智能代理](https://langchain-ai.github.io/langgraphjs/)